<a href="https://colab.research.google.com/github/AlexeyArcher/hello-world/blob/master/test_pretrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Set up env

In [ ]:
import sys
print(sys.version)

3.7.13 (default, Apr 24 2022, 01:04:09) 
[GCC 7.5.0]


In [ ]:
!rm -rf /usr/local/cuda
!ln -s /usr/local/cuda-10.1 /usr/local/cuda

In [ ]:
%%bash
export LD_LIBRARY_PATH=/usr/lib/

In [ ]:
!apt-get install clang-9 llvm-9 llvm-9-dev llvm-9-tools

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  binfmt-support libclang-common-9-dev libclang-cpp9 libclang1-9 libffi-dev
  libllvm9 libomp-9-dev libomp5-9 libpfm4 llvm-9-runtime python-chardet
  python-pkg-resources python-pygments python-yaml python3-pkg-resources
  python3-pygments python3-yaml
Suggested packages:
  clang-9-doc libomp-9-doc llvm-9-doc python-setuptools ttf-bitstream-vera
  python3-setuptools
The following NEW packages will be installed:
  binfmt-support clang-9 libclang-common-9-dev libclang-cpp9 libclang1-9
  libffi-dev libllvm9 libomp-9-dev libomp5-9 libpfm4 llvm-9 llvm-9-dev
  llvm-9-runtime llvm-9-tools python-chardet python-pkg-resources
  python-pygments python-yaml python3-pkg-resources python3-pygments
  python3-yaml
0 upgra

In [ ]:
import subprocess

CUDA_version = [s for s in subprocess.check_output(["nvcc", "--version"]).decode("UTF-8").split(", ") if s.startswith("release")][0].split(" ")[-1]
print("CUDA version:", CUDA_version)

if CUDA_version == "10.0":
    torch_version_suffix = "+cu100"
elif CUDA_version == "10.1":
    torch_version_suffix = "+cu101"
elif CUDA_version == "10.2":
    torch_version_suffix = ""
else:
    torch_version_suffix = "+cu110"

CUDA version: 10.1


In [ ]:
!pip install transformers==3.5.1
!pip install torch==1.7.1{torch_version_suffix} torchvision==0.8.2{torch_version_suffix} torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html
!git clone  https://github.com/sberbank-ai/ru-gpts

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 5.0 MB/s 
     |████████████████████████████████| 1.1 MB 55.9 MB/s 
     |████████████████████████████████| 2.9 MB 47.1 MB/s 
     |████████████████████████████████| 880 kB 63.9 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=39a00a0c995077375b7fab87b343501cc9aa4ce88059959f8dcd5d5759e5f541
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 735.4 MB 16 kB/s 
     |████████████████████████████████| 12.8 MB 52.3 MB/s 
     |████████████████████████████████| 7.6 MB 5.0 MB/s 
  Attempting uninstall: torch
 

In [ ]:
!pip install -U --no-cache-dir gdown --pre

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Download the pretrained model zip

In [ ]:
!gdown https://drive.google.com/uc?id=1IbjnTdbFjkwB71BMDX5DK7kU5hyppr5F

Downloading...
From: https://drive.google.com/uc?id=1IbjnTdbFjkwB71BMDX5DK7kU5hyppr5F
To: /content/11k.zip
100% 1.38G/1.38G [00:11<00:00, 124MB/s]


In [ ]:
!unzip /content/11k.zip

Archive:  /content/11k.zip
   creating: 11k/
  inflating: 11k/config.json         
  inflating: 11k/merges.txt          
  inflating: 11k/optimizer.pt        
  inflating: 11k/pytorch_model.bin   
  inflating: 11k/scheduler.pt        
  inflating: 11k/special_tokens_map.json  
  inflating: 11k/tokenizer_config.json  
  inflating: 11k/training_args.bin   
  inflating: 11k/vocab.json          


Set up the model

In [ ]:
import numpy as np
import torch

In [ ]:
np.random.seed(42)
torch.manual_seed(42)

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [ ]:
tok = GPT2Tokenizer.from_pretrained("11k")

In [ ]:
model = GPT2LMHeadModel.from_pretrained("11k")

In [ ]:
model.cuda()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(2048, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): Laye

Download words dataset from https://github.com/dkulagin/kartaslov/tree/master/dataset/assoc

In [ ]:
!wget https://raw.githubusercontent.com/dkulagin/kartaslov/master/dataset/assoc/assoc.csv
import pandas as pd
df = pd.read_csv('assoc.csv')

--2022-06-10 10:31:35--  https://raw.githubusercontent.com/dkulagin/kartaslov/master/dataset/assoc/assoc.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 59231085 (56M) [text/plain]
Saving to: ‘assoc.csv’

assoc.csv           100%[===================>]  56.49M   214MB/s    in 0.3s    

2022-06-10 10:31:36 (214 MB/s) - ‘assoc.csv’ saved [59231085/59231085]



Good luck

Top k top p (meh)

In [ ]:
sample = df.sample().values[0][0].split(';')
answer = sample[0]
theme = sample[1]

text = "<s>Тема: {} Ответ: {} Вопрос: ".format(theme, answer)
inpt = tok.encode(text, return_tensors="pt")
out = model.generate(inpt.cuda(), max_length=100, repetition_penalty=5.0, do_sample=True, top_k=5, top_p=0.95, temperature=1)

tok.decode(out[0])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'<s>Тема: увлечение Ответ: интересное Вопрос:  В детстве я любил играть в кубки, а еще эти забавные животные были со мной - с краю. </b]\n.<aud : Фильм / Сценарий (сюжет фильма учитывается название сценария) [зачет по названию].    1-й вариант ответа состоит из двух слов о любви Скарлетт и ее свиты... А ДЛЯ ЭТОГО ЧТО ТАКОЕ НАЗВАНИЕ? Наталья Ветлиц'

Beamsearch (better)

In [ ]:
sample = df.sample().values[0][0].split(';')
answer = sample[0]
theme = sample[1]

text = "<s>Тема: {} Ответ: {} ".format(theme, answer)
inpt = tok.encode(text, return_tensors="pt")
out = model.generate(
    inpt.cuda(),
    max_length=50,
    num_beams=10,
    no_repeat_ngram_size=3,
    repetition_penalty=2.,
    num_return_sequences=1, 
)

tok.decode(out[0])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'<s>Тема: норма Ответ: стабильность  Вопрос: ТАКОЕ НАЗВАНИЕ носит период, характеризующийся тем, что в течение установленного промежутка времени значение переменной величины не меняется со временем. </s>\n<'

Режим кастомной генерации (ручной ввод тем и ответов к ним) с номиналами

In [ ]:
def generateTheme(givenTheme, givenAnswer, givenCost):
  text = "<s>Тема: {} Ответ: {} ".format(givenTheme, givenAnswer)
  inpt = tok.encode(text, return_tensors="pt")
  out = model.generate(
    inpt.cuda(),
    max_length=100,
    num_beams=20,
    no_repeat_ngram_size=3,
    repetition_penalty=3.,
    num_return_sequences=4, 
  )
  # print("{}. ".format(givenCost))
  # print(tok.decode(out[0]))
  answer = tok.decode(out[0])
  index = answer.index("Вопрос: ") + 8
  indexEnd = answer.find("</s>")
  if indexEnd == -1 :
    indexEnd = len(answer) - 1
  #indexEnd = answer.index("</s>")
  question = answer[index:indexEnd]
  return(str(givenCost) + ". " + question + "\nОтвет: " + givenAnswer)

In [ ]:
theme = "Анаграммы африканских стран"

answer = ["Нага", "Наугад", "Саунд", "Банго", "Бенни"]
cost = [10, 20, 30, 40, 50]
a = []
a.clear()
for i in range(0,5):
  a.append(generateTheme(theme, answer[i], cost[i]))

print("\n\n" + "Тема: " + theme.upper())
for ai in a:
  print(ai)

a.clear()

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.




Тема: АНАГРАММЫ АФРИКАНСКИХ СТРАН
10. В 2008 году ООН приняла Конвенцию о запрещении или ограничении применения конкретных ЕДИНИЦ ИСПОЛЬЗОВАНИЯ в области футбола. НАГ - одна из самых известных организаций, запрещающих или ограничивающих применение ЭТИХ ДВУХ ФУТБОЛЬНЫХ ИНСТРУМЕНТОВ. 
Ответ: Нага
20. В 2008 году ООН приняла Конвенцию о запрещении или ограничении применения конкретных ЕДИНИЦ ИСПОЛЬЗОВАНИЯ, которые противоречили бы нормам международного права и были направлены на защиту прав человека и гуманнее, чем те, что были ДОЛЖНЫ. 
Ответ: Наугад
30. Ирландия, Бельгия, Нидерланды, Люксембург. Назовите хотя бы две страны, где в футбольном еврокубковом турнире играют друг с другом ТАКОЕ НАЗВАНИЕ и ПОДЗАГОЛОВОК. 
Ответ: Саунд
40. Название ЭТОЙ СТРАНЫ в переводе с санскрита означает невинный, благочестивый. Несложно догадаться, что на жаргоне жителей ЭТОГО ГОСУДАРСТВА слово сакраментальное означает угодный [слуге] (т.е. слуга божьему). 
Ответ: Банго
50. Талисманом чемпионата мира по кри

In [ ]:
def generateQuestion(givenTheme, givenAnswer, givenCost):
  text = "<s>Тема: {} Ответ: {} ".format(givenTheme, givenAnswer)
  inpt = tok.encode(text, return_tensors="pt")
  out = model.generate(
    inpt.cuda(),
    max_length=60,
    num_beams=60,
    no_repeat_ngram_size=2,
    repetition_penalty=2.,
    num_return_sequences=4, 
  )
  print("{}. ".format(givenCost))
  print(tok.decode(out[0]))

In [ ]:
topic = "Шахмат"
answr = "Король (без шута)"
gCost = 10
generateQuestion(topic, answr, gCost)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


10. 
<s>Тема: Шахмат Ответ: Король (без шута)  Вопрос: В шахматах короля обычно изображают в виде ЭТОГО КОЛЛЕКТИВА, а если турнир состоит из одного матча, то его можно использовать и как ЭТОТ ФИЛЬМ. </s
